In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import keras
from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis.preprocessing import decoder_predict
from music_generator.analysis.preprocessing import apply_fourier_on_input
from music_generator.basic.utils import parallel_apply_along_axis
from music_generator.analysis.preprocessing import create_training_data_set, model_predict, apply_fourier_on_dataset

from scipy.io.wavfile import read
from scipy.fftpack import rfft, irfft

import numpy as np
import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def read_file(filename):
    x = read(filename)
    return x[1][:, 0] / 2**15

In [ ]:
bt_0 = read_file("../data/Verplichte Kots_v9.wav")
tot_0 = read_file("../data/Verplichte Kots_v9.wav")

In [ ]:
# play_array(bt_0, range_secs=[20, 24], sample_rate=48000)
# play_array(tot_0, range_secs=[20, 24], sample_rate=48000)

In [ ]:
rel_offset = int(3.135*44100)
x = bt_0[:10*44100 - rel_offset] - tot_0[rel_offset:10*44100]
#play_array(x, range_secs=[0,4])

## Make selection in song

In [ ]:
input = tot_0[rel_offset:]
target = bt_0[:-rel_offset]

# input = input[14*44100:27*44100]

## Define data set

In [ ]:
sample_size = 1024
n_samples = 10240 * 4

In [ ]:
x, y = create_training_data_set(n_samples, sample_size, input, target)

In [ ]:
x, y = map(apply_fourier_on_dataset, (x, y))

In [ ]:
input_shape = x[0].shape[0]
output_shape = x[1].shape[0]

In [ ]:
# https://blog.keras.io/building-autoencoders-in-keras.html

from keras.layers import Dense, Dropout, PReLU, Input
from keras.models import Model
from keras.optimizers import Adam

from keras.regularizers import l2
from keras.models import load_model

In [ ]:
regularizer_config = {} #'kernel_regularizer': l2(l=0),
                      # 'bias_regularizer': l2(l=0)}
model_name = "vpk_ae.h5"

In [ ]:
create_model = True
if create_model:
    encoding_dim = 50

    # Encoder
    input_ = Input((input_shape,))
    _ = input_
    _ = Dense(2000, **regularizer_config)(_)
    _ = PReLU()(_)
    _ = Dense(2000, activity_regularizer=l2(1e-5))(_)
    _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    # _ = Dense(4000, **regularizer_config)(input_)
    # _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    # _ = Dense(4000, **regularizer_config)(_)
    # _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    encoded = Dense(encoding_dim)(_) # , activity_regularizer=l2(1e-4))(_)

    # Decoder
    _ = encoded
    _ = PReLU()(_)
    _ = Dense(2000, activity_regularizer=l2(1e-5))(_)
    _ = PReLU()(_)
    _ = Dense(2000, **regularizer_config)(_)
    _ = PReLU()(_)
    # _ = PReLU()(_)
    # _ = Dropout(0.01)(_)
    # _ = Dense(4000, **regularizer_config)(_)
    # _ = Dropout(0.01)(_)
    # _ = PReLU()(_)
    # _ = Dense(4000, **regularizer_config)(_)
    # _ = Dropout(0.01)(_)
    # _ = PReLU()(_)
    decoded = Dense(output_shape)(_)
    model = Model(input_, decoded)

    model.compile(Adam(lr=0.001), 'mse')
else:
    model = load_model(model_name)

    # create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = encoded_input
# decoder_layer = model.layers[-10](encoded_input)
# decoder_layer = model.layers[-9](decoder_layer)
# decoder_layer = model.layers[-8](decoder_layer)
# decoder_layer = model.layers[-7](decoder_layer)
decoder_layer = model.layers[-6](decoder_layer)
decoder_layer = model.layers[-5](decoder_layer)
decoder_layer = model.layers[-4](decoder_layer)
decoder_layer = model.layers[-3](decoder_layer)
decoder_layer = model.layers[-2](decoder_layer)
decoder_layer = model.layers[-1](decoder_layer)
# create the decoder model
decoder = Model(encoded_input, decoder_layer)

In [ ]:
# play_array(model_predict(model, input, sample_size, rfft, irfft), do_wait_done=False, range_secs=[20, 24])

In [ ]:
# prediction = model.predict(apply_fourier_on_input(input, sample_size))
# prediction = parallel_apply_along_axis(irfft, 0, prediction).reshape(-1)

In [ ]:
# play_array(prediction)
max_duration = len(input) // 44100
prev_loss = np.Infinity
model.save(model_name)
prev_losses = None

In [ ]:
n_to_sample = len(x) // 5

In [ ]:
import keras.backend as K

In [ ]:
K.get_value(model.optimizer.lr)

In [ ]:
while True:
    print(f"Current LR: {K.get_value(model.optimizer.lr)}")
    start = dt.datetime.now()

    losses = 1 / (np.sum((model.predict(x) - x)**2, axis=1) + 1e-5)
    
    if prev_losses is not None:
        loss_diff = (losses - prev_losses)**2
        loss_diff_sig = loss_diff / np.mean(loss_diff)
        p_choose = loss_diff_sig / np.sum(loss_diff_sig)
    else:
        print("Initializing p_choose...")
        p_choose = np.ones(shape=losses.shape)
        p_choose /= np.sum(p_choose)
        
    prev_losses = losses
    ix = np.random.choice(len(x), p=p_choose, size=n_to_sample, replace=False)

    history = model.fit(x[ix], x[ix], epochs=4, verbose=1, batch_size=64) # shuffle=True, steps_per_epoch=64)

    # Save model if improvement
    if history.history['loss'][0] < prev_loss * 1.5:
        model.save(model_name)
        model.optimizer.lr = model.optimizer.lr * (1.01 if history.history['loss'][0] < prev_loss else 0.99)
        prev_loss = history.history['loss'][0]
    else:
        print("Failed to find improvement. Loading previous model...")
        model = load_model(model_name)
        model.optimizer.lr = 0.9*model.optimizer.lr
        
    
    
    # Make prediction
    prediction = model_predict(model, input, sample_size, rfft, irfft)
    time_taken = (dt.datetime.now() - start).seconds
    # Play prediction
    play_array(prediction, range_secs=[0, 0 + min(time_taken, max_duration - 1)], do_wait_done=False)

In [ ]:
history.history['loss'][0]

In [ ]:
# play_array(model_predict(model, input, sample_size), range_secs=[130, 130 + min(time_taken, 250)], do_wait_done=False)

In [ ]:
# play_array(model_predict(model, input, sample_size), range_secs=[20, 24])  

In [ ]:
# play_array(x[0], range_secs=[20,24])
# play_array(target, range_secs=[20,24])

## Test on different file

In [ ]:
test = read_file("../data/Wasmachine_in_a_mineur_v5.wav")
prediction = model_predict(model, test, sample_size, rfft, irfft)
play_array(prediction, range_secs=[0, 0 + min(time_taken, 60)], do_wait_done=False)

In [ ]:
encoder = Model(input_, encoded)
pred = encoder.predict(apply_fourier_on_input(input, sample_size))

In [ ]:
import pandas as pd

In [ ]:
pd.Series(pred.reshape(-1)).hist(bins=51, range=[-2, 2])

In [ ]:
# percentiles = np.percentile(pred, 0, axis=1)
# pred = np.array([np.where(x > p, x, 0) for x, p in zip(pred, percentiles)])

In [ ]:
pred = decoder_predict(decoder, pred, irfft)
pred -= np.mean(pred)

In [ ]:
play_array(pred, range_secs=[10,20])

In [ ]:
%matplotlib inline

from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(np.arange(0, len(pred)), pred[:, 0], pred[:, 1])

In [ ]:
# _ = model.predict(apply_fourier_on_input(input, sample_size))
# play_array(parallel_apply_along_axis(irfft, 0, _).reshape(-1), range_secs=[0,60])

In [ ]:
decoder.predict(x_enc).shape

In [ ]:
x_enc = np.zeros(shape=(10, encoding_dim))

# x_enc = np.array([np.sin(np.arange(0, 2000) / 40) for x in range(0, 10)])

pred = decoder_predict(decoder, x_enc, irfft)

pred -= np.mean(pred)
pred /= (np.percentile(pred, 95) - np.percentile(pred, 5)) * 2

In [ ]:
plt.plot(pred)

In [ ]:
play_array(pred)